In [1]:
%matplotlib
#%matplotlib inline
import os
import csv
import numpy as np
import re
import datetime
import fnmatch
from pathlib import Path
from itertools import groupby
import copy
import itertools
import distance
import nltk
#nltk.download('stopwords')
import pandas as pd
import matplotlib.pyplot as plt
import pickle

from nltk.stem import SnowballStemmer
from nltk.stem import snowball

from itertools import *
from operator import *

# import other jupyter notebooks
import import_ipynb
from openpyxl import load_workbook

Using matplotlib backend: Qt5Agg


In [2]:
# exceptional removal of particular extra sentences not typed by the user 
dict_phraseStim = {
    #'2019-02-05-14-10-39_2ndPart_2' : [1, 2, 3, 4, 5, 6, 9, 10],
    #'2019-01-14-14-58-30' : [0], # ys, session_trial ()
    '2019-01-16-16-36-17_1stPart_2' : [-1], # af_session1
    '2019-01-16-17-00-12_2ndPart_2': [1], # af_session1
    '2019-01-17-15-27-20_1stPart_2' : [4], # Af session2
    '2019-01-17-16-03-27_2ndPart_2' : [0, 1, 2], # Af session2
    '2019-02-06-11-25-41_1' : [7],               # aq_session1    
    '2019-02-08-11-33-53_1stPart_1' : [1],  # aq session3_1_part1
    '2019-02-08-12-11-34_2ndPart_1' : [0, 1, 2, 3],  # aq session3_1_part2
    '2019-01-31-09-37-5_2ndPart_2' : range(1,5), # bh1, session 4 , all sentences except the first one deleted
    '2019-01-31-09-22-49_1stPart_2' : [4],  # bh1_session4_2_part1
    '2019-02-21-16-09-44_1stPart_1' : [1], # bh2_session1
    '2019-02-21-16-22-22_2ndPart_1' : [2, 3, 4],# bh2_session1
    '2019-02-28-17-03-53_1stPart_2' : [2],       # bh2_session3
    '2019-02-28-17-24-2_2ndPart_2' : [0, 2],     # bh2_session3
    '2019-02-14-13-28-20_1stPart_2' : [2], # cw_session3_2_part1
    '2019-02-14-13-57-41_2ndPart_2' : [0, 2, 3], # cw_session3_2_part2
    '2019-02-21-15-01-4_1stPart_1' : [0],        # le_session3
    '2019-02-21-15-25-56_2ndPart_1' : [1],        # le_session3
    '2019-02-18-10-28-35_2' : [0],               # ls2_session4 # picture not described
    '2019-02-05-14-00-27_1stPart_2' : [3],        # mh_session1
    '2019-02-05-14-10-39_2ndPart_2' : [0, 1, 3],   # mh_session1
    '2019-02-08-10-51-3_1stPart_1' : [4],        # mn_session1
    '2019-02-08-11-05-7_2ndPart_1' : [0, 2, 3, 4], # mn_session1
    '2019-02-19-10-34-7_1stPart_1' : [3],          # mn_session3
    '2019-02-19-10-56-43_2ndPart_1' : [1, 2, 3, 4], # mn_session3
    '2019-01-16-15-18-0_1' : [4],            # no_session1
    '2019-02-19-17-10-45_1' : [3],                  # ph_session5
    '2019-01-29-13-25-4_1' : [3],        # ph_session2
    '2019-03-07-16-44-5_2' : [1],                   # rh_session1
    '2019-03-14-13-56-56_2' : [2],                  # rh_session3
    '2019-01-14-15-07-21_1' : [4], # ys_session1
    '2019-01-16-15-18-50_1stPart_1' : [3, 4], # ys_session2
    '2019-01-16-15-42-51_2ndPart_1' : [2], # ys_session2
    '2019-01-30-11-22-25_1' : [3, 5, 7],          # ys_session4
    '2019-01-30-11-22-25_1' : [4, 6, 7] # ys, session 4
}

# exceptional removal of sentences/words typed by the user, but then deleted everything to have a blank scratchpad

dict_phraseUser = {
    "2019-02-06-15-44-15_1" : [2, 3, 6], 
    "2019-02-06-16-19-9_2" : [1, 3, 6, 7],
    "2019-02-12-11-21-21_2" : [0],
    "2019-02-14-14-28-49_1" : [0, 2, 3], # ac_session3_1
    "2019-02-14-14-45-49_2" : [0, 5, 6], # ac_session3_2
    '2019-01-29-14-19-26_1' : [0, 3, 4], # bh1_session2_1
    '2019-01-29-14-40-36_2' : [0, 1, 2], # bh1_session2_2
    '2019-01-30-14-29-29_2' : [4],       # bh1_session3_2
    '2019-01-31-09-12-2_1' : [3],         # bh1_session4_1
    '2019-01-31-09-22-49_1stPart_2' : [4], # bh1_session4_2_part1
    '2019-03-05-09-15-11_1' : [1],         # bh2_session5_1
    '2019-03-05-09-15-11_2' : [1],        # bh2_session5_2
    '2019-02-21-15-55-56_2' : [2],       # ch_session5_2
    '2019-01-30-15-19-36_2' : [1],       # jm_session2_1
    '2019-01-30-15-04-30_1' : [0],         # jm_session2_2
    '2019-01-16-15-18-50_1stPart_1' : [1],  # ys_session2
    '2019-01-16-15-42-51_2ndPart_1' : [0], # ys_session2
    '2019-01-30-11-22-25_1' : [2, 4],       # ys_session4
    '2019-01-30-11-57-3_2' : [0] ,          # ys_session4
    '2019-01-31-13-13-2_1' : [4],           # ys_session5
    '2019-01-30-10-20-32_1' : [0, 1, 2, 3, 4, 5], # no_session4
    '2019-01-30-10-46-38_2' : [0],          # 
    '2019-02-28-17-03-53_1stPart_2' : [2],   # bh2_session3
    '2019-03-12-09-30-5_1' : [0],            # kj_session3
    '2019-02-13-15-20-38_1' : [0, 1, 2, 3, 6], # ls1_session3
    '2019-02-18-10-25-52_1' : [1],              # ls2_session4
    '2019-02-18-10-46-26_2' : [0],            # ls2_session4
    '2019-01-29-13-25-4_1' : [0, 1, 7],        # ph_session2
    '2019-01-29-13-43-50_2' : [0],              # ph_session2
    '2019-03-07-16-17-30_1' : [0],              # rh_session1
    '2019-03-07-16-44-5_2' : [0, 1],         # rh_session1
    '2019-03-14-13-56-56_2' : [0, 1, 3]         # rh_session3
}

# key selection can have extra selections of NextPhrase at the end
dict_keySelectionOfNextPhrase = {
    "2019-02-11-11-18-30_1" : [12, 13], # ac_session1
    "2019-01-16-17-00-12_2ndPart_2" : [12], # af_session1
    "2019-01-17-15-27-20_1stPart_2" : [12], # af_session2
    "2019-02-06-16-19-9_2" : [12], # af_session3
    "2019-02-12-11-07-43_1" : [12], # af_session4
    "2019-02-27-15-08-32_1" : [12], # af_session5
    "2019-01-28-14-30-44_1" : [12], # bh1_session1
    "2019-02-21-16-22-22_2ndPart_1" : [12], # bh2_session1
    "2019-02-18-14-02-56_2" : [12], # le_session1
    "2019-02-19-10-03-14_1" : [12], # le_session2
    "2019-02-08-11-05-7_2ndPart_1" : [12], # mn_session1
    "2019-02-08-11-12-51_2" : [12, 13], # mn_session1
    "2019-02-15-11-38-22_1" : [12, 13], # mn_session2
    "2019-02-15-11-54-25_2" : [12], # mn_session2
    "2019-01-16-15-18-0_1" : [12], # no_session1
    "2019-01-28-13-31-51_1" : [12], # ph_session1
    "2019-01-28-13-49-14_2" : [12], # ph_session1
    "2019-01-14-15-07-21_1" : [12], # ys_session1
    "2019-01-17-15-05-1_1" : [12], # ys_session3
    "2019-01-30-11-22-25_1" : [12], # ys_session4
    "2019-01-31-13-32-2_2" : [12], # ys_session5
}


# key selection when participants skips some sentences
dict_keySelectionNotCompleted = {
    "2019-01-16-16-36-17_1stPart_2" : [0, 1, 3, 5, 7], # af_session1 ---- last sentence is not finished
    "2019-01-16-17-00-12_2ndPart_2" : [0, 1, 3, 4, 5, 7, 9, 11], # af_session1
    "2019-01-17-15-27-20_1stPart_2" : [0, 1, 3, 5, 7, 9, 11], # af_session2 
    "2019-01-17-16-03-27_2ndPart_2" : [0, 1, 2, 3, 4, 5, 6, 7, 9, 11], # af_session2
    "2019-02-08-11-33-53_1stPart_1" : [0, 1, 3, 4, 5, 7, 9, 11], # aq_session3
    "2019-02-08-12-11-34_2ndPart_1" : [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 11], # aq_session3
    "2019-01-28-14-30-44_1" : [0, 1, 3, 5], # bh1_session1
    "2019-01-31-09-22-49_1stPart_2": [0, 1, 3, 5, 7, 9, 10, 11], # bh1_session4
    "2019-01-31-09-37-5_2ndPart_2" : [0, 1, 3, 4, 5, 6, 7, 8, 9, 10, 11], # bh1_session4
    "2019-02-21-16-09-44_1stPart_1" : [0, 1, 3, 4, 5, 7, 9, 11], # bh2_session1
    "2019-02-21-16-22-22_2ndPart_1" : [0, 1, 3, 5, 6, 7, 8, 9, 10, 11], # bh2_session1
    "2019-02-28-17-03-53_1stPart_2" : [0, 1, 3, 5, 6, 7, 9, 11], # bh2_session3
    "2019-02-28-17-24-2_2ndPart_2" : [0, 1, 2, 3, 5], # bh2_session3     ----
    "2019-02-14-13-28-20_1stPart_2" : [0, 1, 3, 5, 6, 7, 9, 11], # cw_session3
    "2019-02-14-13-57-41_2ndPart_2" : [0, 1, 2, 3, 5, 6, 7, 8, 9, 11], # cw_session3
    "2019-02-21-15-01-4_1stPart_1" : [0, 1, 2, 3, 5, 7, 9, 11], # le_session3
    "2019-02-21-15-25-56_2ndPart_1" : [0, 1, 3], # le_session3       ----
    "2019-02-05-14-00-27_1stPart_2" : [0, 1, 3, 5, 7, 8], # mh_session1
    "2019-02-05-14-10-39_2ndPart_2" : [0, 1, 2, 3, 4, 5, 7, 8, 9, 11], # mh_session1
    "2019-02-08-10-51-3_1stPart_1" : [0, 1, 3, 5, 7, 9, 10, 11], # mn_session1
    "2019-02-08-11-05-7_2ndPart_1" : [0, 1, 2, 3, 5, 6, 7, 8, 9, 10, 11], # mn_session1
    "2019-02-19-10-34-7_1stPart_1" : [0, 1, 3, 5, 7, 8, 9, 11], # mn_session3
    "2019-02-19-10-56-43_2ndPart_1" : [0, 1, 3, 4, 5, 6, 7, 8, 9, 10, 11], # mn_session3
    "2019-01-29-13-25-4_1" : [0, 1, 3, 5, 7], # ph_session2  -- sessions where there are less score questions 
    # and more sentences typed
    "2019-01-16-15-18-50_1stPart_1" : [0, 1, 3, 5, 7, 8, 9, 10], # ys_session2
    "2019-01-17-15-05-1_1" : [0, 1, 3, 5],  # ys_session3  -- sessions where there are less score questions 
    # and more sentences typed
    "2019-02-06-11-25-41_1" : [0, 1, 3, 5, 11], # aq_session1 -- sessions where there are less score questions 
    # and more sentences typed
    "2019-01-16-15-42-51_2ndPart_1" : [0, 1, 2, 5], # ys_session2 -- different for reading and writing, this one is for
    # writing
    '2019-01-30-11-22-25_1' : [0, 1, 3, 5, 7, 9, 11]   # ys_session4 -- sessions where there are less score questions 
    # and more sentences typed
   
}

# dictionary for phrase removal just like in the dict_phraseStim, but since not all participants require that, some that 
# do, are added to this new dictionary here
dict_keySelection_phraseStim = {
    '2019-01-17-15-27-20_1stPart_2' : [4], # Af session2
    '2019-01-16-15-18-0_1' : [4],        # no_session1
    '2019-02-19-17-10-45_1' : [3],                  # ph_session5
    '2019-03-07-16-44-5_2' : [1],        # rh_session1
    '2019-03-14-13-56-56_2' : [2],              # rh_session3
    '2019-01-14-15-07-21_1' : [4]         # ys_session1
}


# in the beginning experiments, not everyone started with 800 initial dwell time

dict_dwellTimeOrig_not800 = {
    "2019-01-16-15-51-13_2" : 600, # no_session1
    "2019-01-16-15-18-0_1" : 600, # no_session1
    "2019-01-16-15-43-8_1" : 100, # af_session1
    "2019-01-16-16-36-17_1stPart_2" : 100, # af_session1
    "2019-01-16-17-00-12_2ndPart_2" : 100, # af_session1
    "2019-01-17-15-03-40_1" : 100, # af_session2
    "2019-01-17-15-27-20_1stPart_2" : 0, # af_session2
    "2019-01-17-16-03-27_2ndPart_2" : 100, # af_session2
    "2019-01-14-15-07-21_1" : 500, # ys_session1
    "2019-01-14-15-25-55_2" : 300, # ys_session1
    "2019-01-16-15-18-50_1stpart_1" : 200, # ys_session2
    "2019-01-16-15-42-51_2ndPart_1" : 100, # ys_session2
    "2019-01-16-15-59-55_2" : 100, # ys_session2
    "2019-01-17-15-05-1_1" : 100, # ys_session3
    "2019-01-17-15-31-12_2" : 100 # ys_session3
}


# list of all things that should be present when computing effective time
list_keysToBeCounted = ['Comma', 'BackOne', 'BackMany', 'SpaceBar']

# some sessions do not have gaze data
dict_noGazeData = {
    '2019-01-16-17-00-12_2ndPart_2' : 'no gaze data', # af_session2
    '2019-01-17-15-31-12_2' : 'no gaze data', #ys_session2
    '2019-01-30-11-57-3_2' : 'no gaze data' # ys_session4
}



In [3]:
dict_keySelection_ReadingTrials = {
    "2019-01-16-15-42-51_2ndPart_1" : [0, 1, 3, 5], # ys_session2 
}

dict_keySelection_WritingTrials = {
    "2019-01-16-15-42-51_2ndPart_1" : [0, 1, 2, 5], # ys_session2   
}

dict_keySelection_firstSleepNotCounted = {
    "2019-02-19-10-56-43_2ndPart_1" : 2  # mn_session3 -- 3rd sleep activation is to be counted
}

In [4]:
TimeDwellOrig = 800
TimeFixation = 300

In [5]:
def FixUserKeys(UserKeys_Old):
    # Fix the situation where comma has divided decimals into separate columns
    
    Column_beforeDecimal = [item[2] for item in UserKeys_Old]
    Column_afterDecimal = [item[3] if len(item)>3 else '00' for item in UserKeys_Old]
    
    UserKeys_ProgressPercent = [float(Column_beforeDecimal[i]+'.'+ Column_afterDecimal[i]) for i in 
                                range(0, len(Column_beforeDecimal))]
    UserKeys_Times = [item[0] for item in UserKeys_Old]
    UserKeys_Keys = [item[1] for item in UserKeys_Old]
    
    UserKeys_New = [[UserKeys_Times[ind], UserKeys_Keys[ind], UserKeys_ProgressPercent[ind]] for ind in 
                    range(0, len(UserKeys_ProgressPercent))]
    
    #UserKeys_New = np.concatenate((UserKeys_Times, UserKeys_Keys, UserKeys_ProgressPercent), axis = 0)
    
    
    return UserKeys_New
        

In [6]:
def FixScratchPad(ScratchPad_Old):
    # Fix the situation where comma has divided decimals into separate columns
    
    ScratchPad_Times = [item[0] for item in ScratchPad_Old]
    
    ScratchPad_Phrases = list()
    
    # loop to combine phrases divided by commas
    ScratchPadInd = -1 
    while ScratchPadInd < len(ScratchPad_Old)-1:
        ScratchPadInd = ScratchPadInd + 1
        commasInPhrase = len(ScratchPad_Old[ScratchPadInd])-2
        if commasInPhrase < 1:
            #print(ScratchPad_Old[ScratchPadInd][1])
            ScratchPad_Phrases.append(ScratchPad_Old[ScratchPadInd][1])
            continue
        scratchPadPhrase = ScratchPad_Old[ScratchPadInd][1]
        for phraseJoinNr in range(1, commasInPhrase+1):
            scratchPadPhrase = scratchPadPhrase + ', ' + ScratchPad_Old[ScratchPadInd][1+phraseJoinNr]
        
        ScratchPad_Phrases.append(scratchPadPhrase)
            
        
    ScratchPad_New = [[ScratchPad_Times[ind], ScratchPad_Phrases[ind]] for ind in 
                    range(0, len(ScratchPad_Times))]
    
    #UserKeys_New = np.concatenate((UserKeys_Times, UserKeys_Keys, UserKeys_ProgressPercent), axis = 0)
    
    #print(ScratchPad_New)
    return ScratchPad_New

In [7]:
def FixKeysSelected(KeysSelected_Old):
    # Fix the situation where comma has divided decimals into separate columns
    
    KeysSelected_New = list()
    
    # loop to combine phrases divided by commas
    KeysSelectedInd = -1 
    while KeysSelectedInd < len(KeysSelected_Old)-1:
        KeysSelectedInd = KeysSelectedInd + 1
        
        if KeysSelected_Old[KeysSelectedInd][1].count(',') > 0:
            
            keys_split = KeysSelected_Old[KeysSelectedInd][1].split("\r\n")
            del keys_split[0]
            del keys_split[-1]
            
            keys_split = [key.split(',') for key in keys_split]
            
            KeysSelected_New.extend(keys_split)
        else:
            KeysSelected_New.append(KeysSelected_Old[KeysSelectedInd])
        
    
    return KeysSelected_New

In [8]:
def ComputeDwellTime(userKeys, full_path):
    # modify userKeys to include a column of time instead of progress pct, which is dependent on the then dwell time
    
    TimeDwellOrig = 800
    
    # session name
    session_folder_name = full_path.split('\\')[-1]
    
    if session_folder_name in dic_dwellTimeOrig_not800:
        TimeDwellOrig = dic_dwellTimeOrig_not800[session_folder_name]
    
    #print(TimeDwellOrig)
    
    timeDwell = TimeDwellOrig
    nKey = -1
    for key in userKeys:
        nKey = nKey + 1
        #print(key[1])
        if key[1] == 'IncreaseDwellTime':
            if float(key[2]) == 1:
                timeDwell = timeDwell + 100
        elif key[1] == 'DecreaseDwellTime':
            #print(key[2])
            if float(key[2]) == 1:
                timeDwell = timeDwell - 100
        else:
            userKeys[nKey].append(str(float(key[2])*timeDwell))
    
    return userKeys

In [9]:
def stimPhrasesEdit(PhraseLog, full_path):
   
    # Now extract phrases from the phrase file
    phraseStim_Phrases = [item[1] for item in PhraseLog]
    
    # session name
    session_folder_name = full_path.split('\\')[-1]
        
    phraseStim_PhrasesReduced, phraseStim_timeReduced = zip(*[(x[0], PhraseLog[phraseStim_Phrases.index(x[0])][0]) for x in groupby(phraseStim_Phrases)])
    
    PhraseLogReduced = [[phraseStim_timeReduced[i], phraseStim_PhrasesReduced[i]] for i in range(0, len(phraseStim_PhrasesReduced))]
    
    if PhraseLogReduced[-1][1] == 'THE EXPERIMENT IS NOW DONE':
        del PhraseLogReduced[-1]
        
    if PhraseLogReduced[0][1] == 'phraseText':
        del PhraseLogReduced[0]

    # Here, we want only the sentences typed
    notSentencesToType = list()
    for index in range(0,len(PhraseLogReduced)):
        sentence = PhraseLogReduced[index][1]
        if 'Svar på følgende spørgsmål' in sentence or 'Answer the question:' in sentence or 'What is the complete name of your university?' in sentence or '(give a score between 1 and 7)' in sentence or sentence == '':
            notSentencesToType.append(index)    
    
    for index in sorted(notSentencesToType, reverse=True):
        del PhraseLogReduced[index]
    
    replacingList = []
    PhraseLogReduced = findAndRemoveTrials(session_name, dict_phraseStim, PhraseLogReduced, replacingList)
    
    
    return PhraseLogReduced

In [10]:
# This function will return the datetime in items which is the closest to the date pivot
def nearestTimePoint(dates, date):
    
    for d in dates:
        if d < date:
            nearestTP = d
        else:
            continue
    try: 
        nearestTP
        nearestTPind = dates.index(nearestTP)
    except:
        nearestTP = 0
        nearestTPind = -1
        
    return nearestTP, nearestTPind

In [11]:
# function to convert list of date and time into datetime format list
def timeConversion(timeStrList):
    timeList = list()
    for time in timeStrList:
        time1, t1, t2 = time.partition('+')
        timeList.append(datetime.datetime.strptime(re.sub('[:.T]','-',time1[:-1]), "%Y-%m-%d-%H-%M-%S-%f"))
    return timeList

In [12]:
def OptiKeyTypingTime(UserKeys):
    
    TimeTyping = dict()
    
    time1, t1, t2 = UserKeys[0][0].partition('+')
    startTime = datetime.datetime.strptime(re.sub('[:.T]','-',time1[:-1]), "%Y-%m-%d-%H-%M-%S-%f")
    
    time2, t1, t2 = UserKeys[-1][0].partition('+')
    endTime = datetime.datetime.strptime(re.sub('[:.T]','-',time2[:-1]), "%Y-%m-%d-%H-%M-%S-%f")
    
    TimeTyping['startTime'] = startTime
    TimeTyping['endTime'] = endTime
    
    return TimeTyping

In [13]:
def timeTypingStart(userKeys):
    # From the user keys, find when the user actually starts typing, after having looked at the phrase and all the other 
    # function keys
    
    timeTypingStartInd = 0
    
    timeTypingStartIndList = list()
            
    timeUserTimeInd = 0
    
    ind = 0
    # Get start time of first trial
    
    while ind < len(userKeys):
        #print(len(userKeys[ind][1]))
        if len(userKeys[ind][1]) > 1:
            ind = ind + 1
        else:
            timeTypingStartInd = ind
            timeTypingStartIndList.append(ind)
            break
    
    #print(timeTypingStartInd)
    # Get every next phrase start timings
    while ind < len(userKeys):
        
        if userKeys[ind][1] == 'NextPhrase' and float(userKeys[ind][2]) == 1:
            
            #timeTypingStartIndList.append(ind+1)
            for ind2 in range(ind+1, len(userKeys)):
                if len(userKeys[ind2][1]) > 1:
                    ind = ind + 1
                    continue
                elif userKeys[ind2][1] == 'NextPhrase' and float(userKeys[ind][2]) == 1:
                    ind = ind + 1
                    continue
                else:
                    ind = ind2
                    timeTypingStartIndList.append(ind)
                    break
                    
        else:
            ind = ind + 1
            
    #print(timeTypingStartIndList)
    
    return timeTypingStartIndList

In [14]:
def findAndRemoveTrials(session_name, dictionary_saved, trials, replacingList):
    # function to check the session_name in the dictionary_saved and remove those trials from the dictionary_trial
    
    if session_name in dictionary_saved:
        index_list = dictionary_saved[session_name]
    else:
        index_list = replacingList
    
    
    if index_list:
        if type(trials) == list:
            for index in sorted(index_list, reverse=True):
                del trials[index]
                
        else:
            for index in sorted(index_list, reverse=True):
                del trials['start'][index]
                del trials['end'][index]
        
    return trials    

In [15]:
def gazeConvert2ColumnsTo1(GazeLog, columnIndwValidity_list, indValidity):
    # function to convert pupilsizes from 2 columns for every pupil due to comma use instead of decimal, 
    # to proper pupil sizes
    
    #columnInd_list = [joinColumn1_1, joinColumn1_2, joinColumn2_1, joinColumn2_2]
    
    # number of columns in the final dictionary
    nColumns = int(len(columnIndwValidity_list)/2)
    
    # dictionary of columns that are to be joined later
    columns_beforeDecimal = dict()
    columns_afterDecimal = dict()
    
    # dictionary of joined columns
    columnsFinal = dict()
    
    # dictionary to find and equalize missing values in every column
    missingVal_column = dict()
    missingVal = list()
    
    # find correct index of validity column to be used, to find the actual columns relative to that
    columnsValidity_inUse = list()
    
    for ind, row in enumerate(GazeLog):
        #print(ind)
        #print(sorted(list(np.where(np.array(row) == 'Valid')[0])+list(np.where(np.array(row)=='Invalid')[0]))[indValidity])

        columnsValidity = (sorted(list(np.where(np.array(row) == 'Valid')[0])+list(np.where(np.array(row)=='Invalid')[0]))[indValidity])
        columnsValidity_inUse.append(int(columnsValidity))
    
    columnsValidity_inUse = np.array(columnsValidity_inUse)
    
    columnInd_list = [[columnsValidity_inUse+i] for i in columnIndwValidity_list]
    
    
    for ind in range(0, nColumns):
        
        dict_name = 'column' + str(ind+1)
        columnsFinal[dict_name] = list()
        columns_afterDecimal[dict_name] = list()
                
        #for indItem, item4 in enumerate(GazeLog):
        #    if 'Invalid' not in item4:
        #        if columnInd_list[2*ind+1][0][indItem] < len(item4):
        #            columns_afterDecimal[dict_name].append(item4[columnInd_list[2*ind+1][0][indItem]])
        #        else:
        #            columns_afterDecimal[dict_name].append('0')
        #    else:
        #        columns_afterDecimal[dict_name].append('nan')
        
                
        columns_beforeDecimal[dict_name] = [item4[columnInd_list[2*ind][0][indItem]] if 'Invalid' not in item4 else 'nan' for indItem, item4 in enumerate(GazeLog)]
        columns_afterDecimal[dict_name] = [item4[columnInd_list[2*ind+1][0][indItem]] if 'Invalid' not in item4 and columnInd_list[2*ind+1][0][indItem] < len(item4) else 'nan' for indItem, item4 in enumerate(GazeLog)]

        
        for i in range(0, len(columns_beforeDecimal[dict_name])):
            if 'Valid' not in columns_beforeDecimal[dict_name][i] and 'Valid' not in columns_afterDecimal[dict_name][i]:
                if 'nan' not in columns_beforeDecimal[dict_name][i] and 'nan' not in columns_afterDecimal[dict_name][i]:
                    if float(columns_afterDecimal[dict_name][i]) > 0: 
                        columnsFinal[dict_name].append(float(columns_beforeDecimal[dict_name][i]+'.'+columns_afterDecimal[dict_name][i]))
                    else:
                        columnsFinal[dict_name].append(np.nan)
                else:
                    columnsFinal[dict_name].append(np.nan)
            else:
                # Rarely, the pupil size is a whole number
                columnsFinal[dict_name].append(np.nan) # we will ignore the row, since there is no way of automatically knowing which - 
                # right or left eye has whole number pupil size
    
        missingVal_column[dict_name] = np.argwhere(np.isnan(columnsFinal[dict_name]))
        missingVal_column[dict_name] = list(itertools.chain.from_iterable(missingVal_column[dict_name])) # flatten the list
        
        missingVal.extend(missingVal_column[dict_name])
        
        
    
    missingVal = sorted(set(missingVal))
    
    # if one of the columns are nan, the other one is converted too
    for column in range(0, nColumns):
        dict_name = 'column' + str(column+1)
        for ind in missingVal:
            if ind < len(columnsFinal[dict_name]):
                columnsFinal[dict_name][ind] = np.nan
                
    
    
    
    #print(len(columnsFinal['column1']), len(columnsFinal['column2']))
    
    
    

    return columnsFinal

In [16]:
def filterBlinks(pupilDataL, pupilDataR, timeInDatetime):
    # filter any blinks and nan values lasting around 250ms (on average)
    # first the single nan occurances are replaced with mean of the values on either sides, 
    # as they are assumed to be from hardware problems
    # for the rest of the blinks, 250ms before and after the nan values are interpolated with a linear function
    # returns a dataframe with pupil size, and timestamp
    # http://faculty.washington.edu/chudler/facts.html
   
    
    # create a dataframe from the pupilsize and time
    pupilData_df = pd.DataFrame(list(zip(timeInDatetime, pupilDataL, pupilDataR)), columns=['timeStamp', 'pupilLeft', 'pupilRight'])
    
    # blink is every nan value in the range of 100-400ms 
    # 250 ms (22 samples) before and after the blink will also be removed
    extraBlinkSamples = 22   
    
    
    #pupilData_woSingleMissingData = pupilData.copy()
    #timeList_woSingleMissingData = timeInDatetime.copy()
    #timeInS_woSingleMissingData = timeInS_Trial[-1]
    
    # in case of single missing data, that are due to hardware error, replace with the mean of the pupil size before and
    # after the nan value
    # missing values will be the same for left and right pupil
    missingVal_Single = np.argwhere(np.isnan(pupilDataL))
    missingVal_Single = list(itertools.chain.from_iterable(missingVal_Single)) # flatten the list 
    
    # if no blinks present, return the data
    if len(missingVal_Single) == 0:
        interpolatedNan = np.array([False]*len(pupilData_df))
        return pupilData_df, interpolatedNan
    
    # find the index and values to replace for single nan values
    pupilData_tuples_replaceSingleNan_left = [(val, np.mean([pupilDataL[val-1], pupilDataL[val+1]])) for i, val in enumerate(missingVal_Single) if (val != 0 and val != (len(pupilDataL)-1)) if not np.isnan(pupilDataL[val-1]) and not np.isnan(pupilDataL[val+1])]
    pupilData_tuples_replaceSingleNan_right = [(val, np.mean([pupilDataR[val-1], pupilDataR[val+1]])) for i, val in enumerate(missingVal_Single) if (val != 0 and val != (len(pupilDataR)-1)) if not np.isnan(pupilDataR[val-1]) and not np.isnan(pupilDataR[val+1])]
    
    
    interpolatedNan = np.array([True if ind in dict(pupilData_tuples_replaceSingleNan_left) else False for ind, val in enumerate(pupilDataL)])
    
    
    # replace the single nan values with the mean of the pupil size on either sides
    indList = -1
    for ind, val in pupilData_tuples_replaceSingleNan_left:
        indList = indList + 1
        pupilData_df.iloc[ind, pupilData_df.columns.get_loc('pupilLeft')] = val
        pupilData_df.iloc[ind, pupilData_df.columns.get_loc('pupilRight')] = pupilData_tuples_replaceSingleNan_right[indList][1]
        
    
    # again, find the nan values in the pupil size
    # the list missingVal_SingleDifference contains the index of the first blink, followed by the difference in the index
    # to the next nan value
    
    
    
    # find the nan values again from pupilData['pupilLeft']
    missingVal_Rest_trueFalse = pupilData_df['pupilLeft'].isnull()
    missingVal_Rest = [i for i, x in enumerate(missingVal_Rest_trueFalse) if x]
    
    # if no blinks left, return the current pupilData
    if len(missingVal_Rest) == 0:
        return pupilData_df, interpolatedNan
    
        
    
    
    
    # in the blinks left, find when the blinks start by finding a difference in the consecutive values of the indices
    missingVal_RestDifference = [t - s for s, t in zip(missingVal_Rest, missingVal_Rest[1:])]
    missingVal_RestDifference.insert(0, missingVal_Rest[0])
    
    blinkStart_tupleList = [(ind, sum(missingVal_RestDifference[0:ind+1])) for ind, val in enumerate(missingVal_RestDifference) if val != 1]
    
    blinkStart_tupleList_wLength = list()
    
    # create a list of tuples of blink start index and the length of the blink
    ind = -1
    blinkLengthSum = 0
    for blink_ind, blinkStartInd in blinkStart_tupleList:
        ind = ind + 1
        if ind != len(blinkStart_tupleList) - 1:
            
            blinkLength = blinkStart_tupleList[ind+1][0]-blink_ind
            blinkLengthSum = blinkLengthSum + blinkLength
            
            blinkStart_tupleList_wLength.append(tuple((blinkStartInd, blinkLength)))
        else:
            # for the last blink -- all blink lengths summed and subtracted from the length of the list
            # missingVal_RestDifference 
            blinkLength = len(missingVal_RestDifference)-blinkLengthSum
            blinkStart_tupleList_wLength.append(tuple((blinkStartInd, blinkLength)))
     
    # add to vector with start and end of tuple
    #beforeAfterNan = [False]*len(pupilData_df['pupilLeft'])
    #for blinkStart, blinkLength in blinkStart_tupleList_wLength:
    #    beforeAfterNan[blinkStart] = True
    #    beforeAfterNan[blinkStart+blinkLength] = True
    #    #print('start and end points: ', pupilData_df['timeStamp'][blinkStart], pupilData_df['timeStamp'][blinkStart + blinkLength])
    
    
    # create lists with start and end values for the blinks, based on blinkStart_tupleList_wLength, regardless of the blink length
    blink_missingData_startList = [blinkStartInd - extraBlinkSamples if (blinkStartInd - extraBlinkSamples) > 0 else 0 for blinkStartInd, blinkLength in blinkStart_tupleList_wLength]
    blink_missingData_endList = [blinkStartInd + blinkLength + extraBlinkSamples if (blinkStartInd + blinkLength + extraBlinkSamples) < (len(pupilData_df['pupilLeft'])-1) else (len(pupilData_df['pupilLeft'])-1) for blinkStartInd, blinkLength in blinkStart_tupleList_wLength]
    # create a list of tuples from the start and end points of the blink
    blink_missingData_startEndTuple = [(blinkStart, blink_missingData_endList[ind]) for ind, blinkStart in enumerate(blink_missingData_startList)] 
    
    
    # check if blinks need to be combined - blinksCombine is a list of list of 2 elements, the index of the blinks that should be combined
    blinksCombine = [[ind, ind+1] for ind, blink in enumerate(blink_missingData_startEndTuple[0:-1]) if blink[1] > blink_missingData_startEndTuple[ind+1][0]]
        
    if blinksCombine:
        # combine blinks that need to be combined - if multiple consecutive blinks need to be removed: eg - [1, 2], [2, 3] 
        # are included in the blinksCombine, the combined version should be [1, 3] 
        blinksCombineFinal = list()
        ind = -1
        while ind < len(blinksCombine)-2:
            
            ind = ind + 1
            blinkCombining = blinksCombine[ind]
            blinksCombineFinal.append(blinkCombining)
            while ind < len(blinksCombine)-2 and blinkCombining[1] == blinksCombine[ind+1][0]:
                # change the ending of the last added blink of blinksCombineFinal
                blinksCombineFinal[-1][1] = blinksCombine[ind+1][1]
                ind = ind + 1
            
            
        if len(blinksCombine) == 1:
            blinksCombineFinal = blinksCombine.copy()
            
        
        if blinksCombine[-1][1] != blinksCombineFinal[-1][1]:
            if blinksCombine[-1][0] == blinksCombineFinal[-1][1]:
                blinksCombineFinal[-1][1] = blinksCombine[-1][1]
            else:
                blinksCombineFinal.append(blinksCombine[-1])
        
        
        
        
    #    for w, z in groupby(sorted(list(blinksCombine)), lambda x, y=itertools.count(): next(y)-x):
    #        group = list(z)
    #        blinksCombineFinal.append(tuple((group[0], group[-1])))
        
        for x in sorted(blinksCombineFinal, reverse=True):
            new_start = blink_missingData_startEndTuple[x[0]][0] 
            new_end = blink_missingData_startEndTuple[x[1]][1] 
            
            x_start = x[0]
            x_end = x[1]
            
            # delete also the blinkStart_tupleList_wLength, since it is going to be used to compute other metrics
            for blinkRemove in range(x[1], x[0]-1, -1):
                del blink_missingData_startEndTuple[blinkRemove]
            
            blink_missingData_startEndTuple.insert(x[0], tuple((new_start, new_end)))
    
    
    #blinkAndNonBlinkDurationList = [length/90 for start, length in blinkStart_tupleList_wLength]
    #timeInS_Trial_filter = timeInS_Trial[-1] - sum(blinkAndNonBlinkDurationList) 
    
    
    # remove blinks from data
    for blinkStart, blinkEnd in blink_missingData_startEndTuple:
        pupilData_df.loc[blinkStart:blinkEnd,'pupilLeft'] = np.nan
        pupilData_df.loc[blinkStart:blinkEnd,'pupilRight'] = np.nan
        replaceTrueList = range(blinkStart, blinkEnd+1, 1)
        interpolatedNan[replaceTrueList] = True
    
    
    
    pupilData_df['pupilLeft'] = pupilData_df['pupilLeft'].astype(float).interpolate('linear', limit_direction = 'both')
    pupilData_df['pupilRight'] = pupilData_df['pupilRight'].astype(float).interpolate('linear', limit_direction = 'both')
    
    if pupilData_df.isnull().any().any():
        print('nan values in filtered data')
        #for i,val in enumerate(pupilData_filter[0:5000]):
        #    print(i, val, pupilData_woSingleMissingData[i])
        
    
    return pupilData_df, interpolatedNan

In [17]:
def hampel(dvec, radius=5, nsigma=3, rem_nomed=False):

    # replace outliers with median values (hampel filter)
    
    mvec = pd.Series(dvec).rolling(radius*2+1, center=True, min_periods=radius).median()
    svec = 1.4862 * np.abs(dvec-mvec).rolling(radius*2+1, center=True, min_periods=radius).median()
    plonk = np.abs(dvec-mvec) > nsigma*svec
    dvec = np.array(dvec)
    dvec[plonk.tolist()] = mvec[plonk.tolist()]

    # remove "bad data" where we cannot calculate a median value due to already missing values
    if (rem_nomed):
        dvec[np.isnan(mvec)] = np.nan
    return dvec

In [18]:
def weightedMean(pupilData_df, window):
    
    # find the rolling standard deviations
    std_left_end = pupilData_df['pupilLeft'].rolling(window).std()[window-1:]
    std_right_end = pupilData_df['pupilRight'].rolling(window).std()[window-1:]
    
    std_left = [np.std(pupilData_df['pupilLeft'][0:i]) for i in range(1,window-1)]
    std_left.insert(0,0)
    
    
    std_right = [np.std(pupilData_df['pupilRight'][0:i]) for i in range(1,window-1)]
    std_right.insert(0,0)
    
    
        
    std_left.extend(std_left_end)
    std_right.extend(std_right_end)
    
    
    # if std_left is 0, the succeeding value is taken
    i = len(std_left)-1
    while i > -1:
        left = std_left[i]
        if left == 0:
            if i < len(std_left)-1:
                std_left[i] = std_left[i+1]
            else:
                std_left[i] = 2 # to give 0.5 for 1/std
        i = i - 1
    
    # if std_left is 0, the succeeding value is taken
    i = len(std_right) - 1
    while i > -1:
        right = std_right[i]
        if right == 0:
            if i < len(std_right)-1:
                std_right[i] = std_right[i+1]
                
            else:
                std_right[i] = 2 # to give 0.5 for 1/std
        i = i - 1
    
    # coefficients are 1/std
    coeff_left_notNormalized = 1/np.array(std_left)
    coeff_right_notNormalized = 1/np.array(std_right)
    
    coeff_left = coeff_left_notNormalized/(coeff_left_notNormalized+coeff_right_notNormalized)
    coeff_right_a = coeff_right_notNormalized/(coeff_left_notNormalized+coeff_right_notNormalized)
    coeff_right = 1-coeff_left
    
    #for i, x in enumerate(coeff_left):
    #    print(coeff_left_notNormalized[i], x, coeff_right_notNormalized[i], coeff_right[i])
    
    
    #coeff_left_notNormalized = [1/std if not np.isnan(std) and std!=0 else 0.5 for std in std_left]
    #coeff_right_notNormalized = [1/std if not np.isnan(std) and std!=0 else 0.5 for std in std_right]
    
    #coeff_left = np.array([left/(left+coeff_right_notNormalized[ind]) for ind, left in enumerate(coeff_left_notNormalized)])
    #coeff_right = 1-coeff_left
    
    pupilData_df['pupilMean_weighted'] = coeff_left*pupilData_df['pupilLeft'] + coeff_right*pupilData_df['pupilRight']
    pupilData_df['pupilMean'] = 0.5*pupilData_df['pupilLeft'] + 0.5*pupilData_df['pupilRight']
    
    #error = [np.abs(np.sum(pupilData_df['pupilLeft'][ind:ind+window-1])-np.sum(pupilData_df['pupilRight'][ind:ind+window-1]))/window for ind, left in enumerate(pupilData_df['pupilLeft'])]
    
    
    #print(len(pupilData_df['pupilLeft']), len(error))
    
    #print(error[0:20])
    #print(error[-20:])
    
    print(np.corrcoef(pupilData_df['pupilLeft'], pupilData_df['pupilRight']))
    
    corr = np.corrcoef(pupilData_df['pupilLeft'], pupilData_df['pupilRight'])[0][1]
    
    
    #error = np.abs(pupilData_df['pupilLeft'] - pupilData_df['pupilRight'])
    
    #plt.figure()
    #plt.plot(error)
    #plt.plot(pupilData_df['pupilLeft'])
    #plt.plot(pupilData_df['pupilRight'])
    
    
    
    
    return pupilData_df, corr

In [19]:
def FilterPupilSize_wWeightedMean(GazeLog, TimeTyping, subjectAndSessionName):
    # function that uses the list of start and end trial times to find the pupil sizes for those trials and plots them
    
    # first create a list of times in gaze log
    timeStrGazeLog = [item3[0] for item3 in GazeLog]
    # convert the list of strings to datetime formats
    timeGazeLog = timeConversion(timeStrGazeLog)
    
    # internal time, to depict seconds
    timeInternalGazeLog = [float(item3[1]) for item3 in GazeLog]
    
    # extract pupil sizes in decimals from the strange 2 columns for every pupil
    pupil_indWrtValidityL = [1, 2]
    pupil_validityL = 4
    pupilLogL_raw = gazeConvert2ColumnsTo1(GazeLog, pupil_indWrtValidityL, pupil_validityL)
    
    pupil_indWrtValidityR = [1, 2]
    pupil_validityR = 5
    pupilLogR_raw = gazeConvert2ColumnsTo1(GazeLog, pupil_indWrtValidityR, pupil_validityR)
    
    
    # reduce the data to start and end of typing time
    timeTyping_start, timeTyping_startInd = nearestTimePoint(timeGazeLog, TimeTyping['startTime'])
    timeTyping_end, timeTyping_endInd = nearestTimePoint(timeGazeLog, TimeTyping['endTime'])
    
    pupilLogL_wDefinedTime = pupilLogL_raw['column1'][timeTyping_startInd:timeTyping_endInd+1]
    pupilLogR_wDefinedTime = pupilLogR_raw['column1'][timeTyping_startInd:timeTyping_endInd+1]
    
    timeGazeLog_wDefinedTime = timeGazeLog[timeTyping_startInd:timeTyping_endInd+1]
    
    timeInS_GazeLog_wDefinedTime = timeInternalGazeLog[timeTyping_startInd:timeTyping_endInd+1]
    timeInS_Difference = [(t - s)/1000000 for s, t in zip(timeInS_GazeLog_wDefinedTime, timeInS_GazeLog_wDefinedTime[1:])]
    timeInS_Difference.insert(0, 0)
    
    
    #timeInS = [sum(timeInS_Difference[:i]) for i, v in enumerate(timeInS_Difference)]
    
    pupilData_df, interpolated_items = filterBlinks(pupilLogL_wDefinedTime, pupilLogR_wDefinedTime, timeGazeLog_wDefinedTime)
    
    
    
    
    #timeGazeLog_plot = np.arange(0, timeInS[-1], 1/90)
    
    #plotPupilSize_checkFilter(pupilData_df, pupilLogL_wDefinedTime, blinkStartAndEnd, 'blink removal', subjectAndSessionName)
    
    
    
    
    pupilData_df_hampel = dict()
    
    pupilData_df_hampel = pupilData_df.copy()
    pupilData_df_hampel['pupilLeft'] = hampel(pupilData_df['pupilLeft'], 25, 3, False)
    pupilData_df_hampel['pupilRight'] = hampel(pupilData_df['pupilRight'], 25, 3, False)
        
    
    # weighted mean of right and left pupils, based on their standard deviation 
    std_window = 25
    pupilDataMean_df, correlationRL = weightedMean(pupilData_df_hampel, std_window)
    
    
        
    return pupilDataMean_df, interpolated_items, correlationRL

In [20]:
def FindTrialTimes(KeysSelected, timeTyping, full_path):
    # function to find start and end of tasks in experiments
    
    # session name
    session_folder_name = full_path.split('\\')[-1]
    
    timeTrialDict = dict()
    timeTrialDict = {'start': [],
                    'end':[]}
    
    nTrial = -1
    
    for keys in KeysSelected:
        
            
        
        if keys[1] == 'NextPhrase':
            nTrial = nTrial + 1
            time1, t1, t2 = keys[0].partition('+')
            endTimeTrial = datetime.datetime.strptime(re.sub('[:.T]','-',time1[:-1]), "%Y-%m-%d-%H-%M-%S-%f")
            
            if nTrial != 0:
                # print('end: ', endTimeTrial)
                #print('')
                timeTrialDict['end'].append(endTimeTrial)
            
            
            # add 5s for the start time of the next phrase
            
            time1, t1, t2 = keys[0].partition('+')
            startTimeTrial_wo5s = datetime.datetime.strptime(re.sub('[:.T]','-',time1[:-1]), "%Y-%m-%d-%H-%M-%S-%f")
            
            startTimeTrial = startTimeTrial_wo5s + datetime.timedelta(seconds=5)
            
            #print('start: ', startTimeTrial)
            timeTrialDict['start'].append(startTimeTrial)
        
    del timeTrialDict['start'][-1]
    
    
    # remove the extra selections of NewPhrase at the end of some sessions
    replacingList = []
    timeTrialDict = findAndRemoveTrials(session_folder_name, dict_keySelectionOfNextPhrase, timeTrialDict, replacingList)
    
    timeTrialDict_copy = copy.deepcopy(timeTrialDict)
    
    # separate the reading and writing trials for some participants who read in the actual trial, but write in the next
    # trial
    if session_folder_name in dict_keySelection_ReadingTrials:
        # check the reading and writing separate dictionaries
        #print('reading and writing sessions are separate')
        
        #print(len(timeTrialDict['start']))
        # writing trials - 
        timeTrialDict_writing = findAndRemoveTrials(session_folder_name, dict_keySelection_WritingTrials, timeTrialDict, replacingList)
        #print(len(timeTrialDict_copy['start']))
        
        # reading trials
        timeTrialDict_reading = findAndRemoveTrials(session_folder_name, dict_keySelection_ReadingTrials, timeTrialDict_copy, replacingList)
    else:
        # some participants skip some sentences, and then it affects the scoreQuestions too. Remove the skipped sentences or 
        # remove the score questions 
        # for these participants, reading and writing trials are the same
        
        #print('same reading and writing trials')
        scoreQuestions = [0, 1, 3, 5, 7, 9, 11]
        timeTrialDict = findAndRemoveTrials(session_folder_name, dict_keySelectionNotCompleted, timeTrialDict, scoreQuestions)
        
        # most of the skipped sentences are removed, but for those that are not removed
        timeTrialDict_writing = findAndRemoveTrials(session_folder_name, dict_keySelection_phraseStim, timeTrialDict, replacingList)
        
        timeTrialDict_reading = timeTrialDict_writing 
        
            
    return timeTrialDict_reading, timeTrialDict_writing

In [21]:
def DivideTimeIntoTrials(PupilData, TimeEpochTrial):
    # use the dictionary of TimeEpochTrial to find the divide between trials
    
    EventTrials = dict()
    EventTrials['start'] =  [False]*len(PupilData['pupilLeft'])
    EventTrials['end'] = [False]*len(PupilData['pupilLeft'])
    
    
    for trialNr in range(0, len(TimeEpochTrial['start'])):
        
        # find start and end time in gazeLog
        timeStart_trial, timeStartInd_trial = nearestTimePoint(PupilData['timeStamp'].tolist(), TimeEpochTrial['start'][trialNr])
        timeEnd_trial, timeEndInd_trial = nearestTimePoint(PupilData['timeStamp'].tolist(), TimeEpochTrial['end'][trialNr])
        
        EventTrials['start'][timeStartInd_trial] = True
        EventTrials['end'][timeEndInd_trial] = True
        
    EventTrials_index = dict()
    EventTrials_index['start'] = [i for i, x in enumerate(EventTrials['start']) if x] 
    EventTrials_index['end'] = [i for i, x in enumerate(EventTrials['end']) if x] 
            
    return EventTrials, EventTrials_index
    

In [22]:
def FindReadingPartsOfTrial(EventTrials_reading, KeysSelected_new, PupilSize_df, full_path):
    
    # session name
    session_folder_name = full_path.split('\\')[-1]
    
    KeysSelected_timeStr = [key[0] for key in KeysSelected_new]
    KeysSelected_time = timeConversion(KeysSelected_timeStr)
    
    KeysSelected_keys = [key[1] for key in KeysSelected_new]
    
    EventReading = dict()    
    EventReading['start'] = list()
    EventReading['end'] = list()
    
    EventReading_index = dict()    
    EventReading_index['start'] = list()
    EventReading_index['end'] = list()
    
    EventBaseline_startKeyTime = list()
    
    for ind, startTimeInd in enumerate(EventTrials_reading['start']):
        startTimeTrial = PupilSize_df['timeStamp'][startTimeInd]
        endTimeTrial = PupilSize_df['timeStamp'][EventTrials_reading['end'][ind]]
        
        # reading start is the same as trial start
        EventReading['start'].append(startTimeTrial)
        EventReading_index['start'].append(startTimeInd)
        
        # start and end time of trial, with respect to the keysSelected
        startTrial_keysTime, startTrial_keysInd = nearestTimePoint(KeysSelected_time, startTimeTrial)
        endTrial_keysTime, endTrial_keysInd = nearestTimePoint(KeysSelected_time, endTimeTrial)
        
        EventBaseline_startKeyTime.append(startTrial_keysTime)
        
        # create keysSelected_trial - keys selected in the trial, to make it easier to find the Sleep button selection 
        # after the trial start
        keysSelected_trial = KeysSelected_keys[startTrial_keysInd:endTrial_keysInd]
        
        # compute the end of reading time -- which is when the sleep button is selected, most of the times
        if session_folder_name not in dict_keySelection_firstSleepNotCounted:
            endReading_gazeTime, endReading_gazeInd = nearestTimePoint(PupilSize_df['timeStamp'].tolist(), KeysSelected_time[startTrial_keysInd + keysSelected_trial.index('Sleep')])
            
        else:
            sleepActivated_allInd = [ind for ind, val in enumerate(keysSelected_trial) if 'Sleep' in val][dict_keySelection_firstSleepNotCounted[session_folder_name]]
            print(keysSelected_trial[sleepActivated_allInd], KeysSelected_time[startTrial_keysInd+sleepActivated_allInd])
            endReading_gazeTime, endReading_gazeInd = nearestTimePoint(PupilSize_df['timeStamp'].tolist(), KeysSelected_time[startTrial_keysInd + sleepActivated_allInd])
            
        #print(KeysSelected_keys[startTrial_keysInd + keysSelected_trial.index('Sleep')], KeysSelected_time[startTrial_keysInd + keysSelected_trial.index('Sleep')], endReading_gazeTime, PupilSize_df['timeStamp'][endReading_gazeInd+1])
        
        # add the end time of reading, but the next element after the sleep selection matches gaze, to allow complete 
        # sleep selection
        
        EventReading['end'].append(PupilSize_df['timeStamp'][endReading_gazeInd+1])
        EventReading_index['end'].append(endReading_gazeInd+1)
        
        #print(ind)
        #print('reading: ', EventReading['start'][ind-1], EventReading['end'][ind])
        
    return EventReading, EventReading_index
    

In [23]:
def FindWritingPartsOfTrial(EventTrials_writing, KeysSelected_new, PupilSize_df, EventReading_index):
    
    KeysSelected_timeStr = [key[0] for key in KeysSelected_new]
    KeysSelected_time = timeConversion(KeysSelected_timeStr)
    
    KeysSelected_keys = [key[1] for key in KeysSelected_new]
    
    EventWriting = dict()    
    EventWriting['start'] = list()
    EventWriting['end'] = list()
    
    EventWriting_index = dict()    
    EventWriting_index['start'] = list()
    EventWriting_index['end'] = list()
    
    for ind, startTimeInd in enumerate(EventTrials_writing['start']):
        startTimeTrial = PupilSize_df['timeStamp'][startTimeInd]
        
        endTimeTrial = PupilSize_df['timeStamp'][EventTrials_writing['end'][ind]]
        
        endTimeReading = PupilSize_df['timeStamp'][EventReading_index['end'][ind]]
        
        #print(endTimeReading)
        
        # for some participants, reading and writing trials are different. So their writing times will not be the end of 
        # the reading time.
        # Regardless, the writing time should start later than when the reading time ends.
        # We choose the starting time for writing as the one that is later than the start time from writing trials
        # and end time from reading trials
        
        if startTimeTrial > endTimeReading:
            EventWriting['start'].append(startTimeTrial)
            EventWriting_index['start'].append(startTimeInd)
        else:
            EventWriting['start'].append(endTimeReading)
            EventWriting_index['start'].append(EventReading_index['end'][ind])
        
        EventWriting['end'].append(endTimeTrial)
        EventWriting_index['end'].append(EventTrials_writing['end'][ind])
        
        #print(ind)
        #print('writing: ', EventWriting['start'][ind], EventWriting['end'][ind])
        
    
    
    return EventWriting, EventWriting_index     

In [24]:
def GetBaseline(EventTrials_reading, PupilSize_df, NanValues, TimeBaseline):
    
    timeBaselineCheck = 5 # seconds
    
    pupilSize_baseline = dict()
    pupilSize_baseline['pupilLeft'] = list()
    pupilSize_baseline['pupilRight'] = list()
    
    for ind, startTimeInd in enumerate(EventTrials_reading['start']):
        startBaselineTime_NextPhraseEnds = PupilSize_df['timeStamp'][startTimeInd]
        StartBaselineTime, StartBaselineInd = nearestTimePoint(PupilSize_df['timeStamp'].tolist(), \
                startBaselineTime_NextPhraseEnds-datetime.timedelta(seconds=TimeBaseline))
        startBaselineTime_w5s = startBaselineTime_NextPhraseEnds + datetime.timedelta(seconds=timeBaselineCheck)
        
        EndBaselineTime, EndBaselineInd = nearestTimePoint(PupilSize_df['timeStamp'].tolist(), startBaselineTime_w5s)
        
        trial_pupilSize = PupilSize_df['timeStamp'][StartBaselineInd:EndBaselineInd]
        trial_nanValues = NanValues[StartBaselineInd:EndBaselineInd]
        
        # find baseline pupil size, for TimeBaseline 
        overlapping_time = 0.1 #s
        sampling_frequency = 90
        nIterations = int((timeBaselineCheck+TimeBaseline)/overlapping_time)+1
        iteration = 0
        done = 0
        while iteration < nIterations:
            NanValuesPresent = trial_nanValues[int(iteration*(TimeBaseline-overlapping_time)*sampling_frequency):\
                                     int((iteration*(TimeBaseline-overlapping_time) + TimeBaseline)*sampling_frequency)]
            
            if sum(NanValuesPresent) < 1 and len(NanValuesPresent)>1:
                #print('done')
                baselineStartInd = StartBaselineInd + int(iteration*(TimeBaseline-overlapping_time)*sampling_frequency)
                baselineEndInd = StartBaselineInd + int((iteration*(TimeBaseline-overlapping_time) + TimeBaseline)*\
                                                        sampling_frequency)
                done = 1
                
                #print(np.mean(PupilSize_df['pupilLeft'][baselineStartInd:baselineEndInd]))
                #print(np.mean(PupilSize_df['pupilRight'][baselineStartInd:baselineEndInd]))
                
                pupilSize_baseline['pupilMean_weighted'].append(np.mean(PupilSize_df['pupilMean_weighted'][baselineStartInd:baselineEndInd]))
                
                
                #plt.figure()
                #plt.plot(PupilSize_df['timeStamp'][baselineStartInd:baselineEndInd], PupilSize_df['pupilLeft'][\
                #    baselineStartInd:baselineEndInd])
                #plt.plot(PupilSize_df['timeStamp'][baselineStartInd:baselineEndInd], PupilSize_df['pupilRight'][\
                #    baselineStartInd:baselineEndInd])
                
                break
            
            iteration = iteration + 1
            
        if done < 1:
            print('baseline not found')
            # add another 5s to the baseline search
            
            extraBaseLineEndTime, extraBaseLineEndInd = nearestTimePoint(PupilSize_df['timeStamp'].tolist(), EndBaselineTime\
                    +datetime.timedelta(seconds=timeBaselineCheck))
            
            trial_nanValues = NanValues[EndBaselineInd:extraBaseLineEndInd]
            
            nIterations = int((timeBaselineCheck+TimeBaseline)/overlapping_time)+1
            iteration = 0
            done = 0
            while iteration < nIterations:
                NanValuesPresent = trial_nanValues[int(iteration*(TimeBaseline-overlapping_time)*sampling_frequency):\
                                     int((iteration*(TimeBaseline-overlapping_time) + TimeBaseline)*sampling_frequency)]
            
                if sum(NanValuesPresent) < 1 and len(NanValuesPresent)>1:
                    #print('done')
                    baselineStartInd = EndBaselineInd + int(iteration*(TimeBaseline-overlapping_time)*sampling_frequency)
                    baselineEndInd = EndBaselineInd + int((iteration*(TimeBaseline-overlapping_time) + TimeBaseline)*\
                                                        sampling_frequency)
                    done = 1
                
                    #print(np.mean(PupilSize_df['pupilLeft'][baselineStartInd:baselineEndInd]))
                    #print(np.mean(PupilSize_df['pupilRight'][baselineStartInd:baselineEndInd]))
                
                    pupilSize_baseline['pupilLeft'].append(np.mean(PupilSize_df['pupilLeft'][baselineStartInd:baselineEndInd]))
                    pupilSize_baseline['pupilRight'].append(np.mean(PupilSize_df['pupilRight'][baselineStartInd:baselineEndInd]))
        
                
                    #plt.figure()
                    #plt.plot(PupilSize_df['timeStamp'][baselineStartInd:baselineEndInd], PupilSize_df['pupilLeft'][\
                    #    baselineStartInd:baselineEndInd])
                    #plt.plot(PupilSize_df['timeStamp'][baselineStartInd:baselineEndInd], PupilSize_df['pupilRight'][\
                    #    baselineStartInd:baselineEndInd])
                
                    break
            
                iteration = iteration + 1
            
            if done < 1:
                print('baseline not found -- again')
                
            
            
    return pupilSize_baseline

In [25]:
class DataForEveryTrial:
    subjectID = ''
    blockNumber = ''
    sessionNumber = ''
    variable = ''
    dataForTrial = ''
    resultPathName = ''
   
    
    def printInfo(self):
        dataFrame = pd.DataFrame(list(zip([self.subjectID]*len(self.dataForTrial), [self.blockNumber]*len(self.dataForTrial), [self.sessionNumber]*len(self.dataForTrial), range(0,len(self.dataForTrial)+1), self.dataForTrial)), columns=['subjectID', 'block', 'session', 'trial', self.variable])
        
        return dataFrame
    
    def AddToFile(self):
        
        dataFrame = pd.DataFrame(list(zip([self.subjectID]*len(self.dataForTrial), [self.blockNumber]*len(self.dataForTrial), [self.sessionNumber]*len(self.dataForTrial), range(0,len(self.dataForTrial)+1), self.dataForTrial)), columns=['subjectID', 'block', 'session', 'trial', self.variable])
        book = load_workbook(self.resultPathName)
        writer = pd.ExcelWriter(self.resultPathName, engine='openpyxl')
        writer.book = book
        writer.sheets = dict((ws.title, ws) for ws in book.worksheets)
        startrow = writer.sheets['Sheet1'].max_row
        dataFrame.to_excel(writer, startrow = startrow, index = False, header = False)
        
        writer.save()
    

In [26]:
def getBaselineQuestion(TimeTyping, KeysSelected_new):

    KeysSelected = [key[1] for key in KeysSelected_new]
    
    BaselineQuestionEndInd = KeysSelected.index('NextPhrase')
    
    time1, t1, t2 = KeysSelected_new[BaselineQuestionEndInd][0].partition('+')
    BaselineQuestionEndTime = datetime.datetime.strptime(re.sub('[:.T]','-',time1[:-1]), "%Y-%m-%d-%H-%M-%S-%f")
    
    
    return (TimeTyping['startTime'], BaselineQuestionEndTime)

In [27]:
def GetBaselineForEveryTrial(EventTrials_reading, PupilSize_df, NanValues, TimeBaseline, TimeBaselineQuestion, \
                      SubjectAndSessionName):

    subjectID = SubjectAndSessionName.split('__')[0]
    timeBaselineCheck = 5 # seconds
    
    pupilSize_baseline = list()
    
    for ind, startTimeInd in enumerate(EventTrials_reading['start']):
        startBaselineTime_NextPhraseEnds = PupilSize_df['timeStamp'][startTimeInd]
        
        StartBaselineTime, StartBaselineInd = nearestTimePoint(PupilSize_df['timeStamp'].tolist(), \
                startBaselineTime_NextPhraseEnds-datetime.timedelta(seconds=TimeBaseline))
        
        EndBaselineTime, EndBaselineInd = nearestTimePoint(PupilSize_df['timeStamp'].tolist(), StartBaselineTime + \
                                                           datetime.timedelta(seconds=timeBaselineCheck))
        
        
        
        trial_nanValues = NanValues[StartBaselineInd:EndBaselineInd]
        
        # find baseline pupil size, for TimeBaseline 
        overlapping_time = TimeBaseline/2 #s
        sampling_frequency = 90
        nIterations = int((timeBaselineCheck+TimeBaseline)/(TimeBaseline-overlapping_time))+1
        iteration = 0
        done = 0
        
        while not done:
            
            iteration = 0
            while iteration < nIterations:
                #print(iteration, StartBaselineInd + int(iteration*(TimeBaseline-overlapping_time)*sampling_frequency),\
                #     StartBaselineInd + int((iteration*(TimeBaseline-overlapping_time) + TimeBaseline)*\
                #                                        sampling_frequency))
                NanValuesPresent = trial_nanValues[int(iteration*(TimeBaseline-overlapping_time)*sampling_frequency):\
                                     int((iteration*(TimeBaseline-overlapping_time) + TimeBaseline)*sampling_frequency)]
                
                if sum(NanValuesPresent) < 1 and len(NanValuesPresent)>1:
                    #print(ind, 'done')
                    baselineStartInd = StartBaselineInd + int(iteration*(TimeBaseline-overlapping_time)*sampling_frequency)
                    baselineEndInd = StartBaselineInd + int((iteration*(TimeBaseline-overlapping_time) + TimeBaseline)*\
                                                        sampling_frequency)
                    done = 1
                
                    pupilSize_baseline.append(np.mean(PupilSize_df['pupilMean_weighted'][baselineStartInd:\
                                                                                             baselineEndInd]))
                    
                    break
            
                iteration = iteration + 1
            
            extraBaseLineEndTime, extraBaseLineEndInd = nearestTimePoint(PupilSize_df['timeStamp'].tolist(), \
                    EndBaselineTime+datetime.timedelta(seconds=timeBaselineCheck))
            
            trial_nanValues = NanValues[EndBaselineInd:extraBaseLineEndInd]
            StartBaselineInd = EndBaselineInd
            EndBaselineTime = extraBaseLineEndTime
            EndBaselineInd = extraBaseLineEndInd
            
            #print('shift baseline')
    
    return pupilSize_baseline

In [28]:
def GetPupilSize_startingAndEnding(EventTrialsInd, PupilMean_df, TrialComputationTime, NanValues):
    
    pupilSize_TrialStart = list()
    pupilSize_TrialEnd = list()
    
    sampling_frequency = 90
    TrialComputationCheckTime = 5
    TrialComputationCheckInd = int(TrialComputationCheckTime*sampling_frequency)
    TrialComputationInd = int(TrialComputationTime*sampling_frequency)
    
    for ind, trialStart_startInd in enumerate(EventTrialsInd['start']):
        
        # starting pupil size
        
        trialStart_endInd = trialStart_startInd + TrialComputationCheckInd
        trial_nanValues = NanValues[trialStart_startInd:trialStart_endInd]
        
        overlapping_time = TrialComputationTime/2 #s
        nIterations = int((TrialComputationCheckTime)/(TrialComputationTime-overlapping_time))-1
        iteration = 0
        done = 0
        
        
        while not done:
            
            iteration = 0
            while iteration < nIterations:
                NanValuesPresent = trial_nanValues[int(iteration*(TrialComputationTime-overlapping_time)*\
                        sampling_frequency):int((iteration*(TrialComputationTime-overlapping_time) + \
                                                 TrialComputationTime)*sampling_frequency)]
                
                if sum(NanValuesPresent) < 1 and len(NanValuesPresent)>1:
                    
                    TrialStart_noNanStartInd = trialStart_startInd + int(iteration*(TrialComputationTime-overlapping_time)*\
                                                                    sampling_frequency)
                    TrialStart_noNanEndInd = trialStart_startInd + int((iteration*(TrialComputationTime-overlapping_time) + \
                                                    TrialComputationTime)*sampling_frequency)
                    done = 1
                
                    pupilSize_TrialStart.append(np.mean(PupilMean_df['pupilMean_weighted'][TrialStart_noNanStartInd:\
                                                                                             TrialStart_noNanEndInd]))
                    break
            
                iteration = iteration + 1
            
            
            extraTrialStart_endInd = trialStart_endInd + TrialComputationCheckInd
            
            trial_nanValues = NanValues[trialStart_endInd:extraTrialStart_endInd]
            trialStart_endInd = extraTrialStart_endInd
            trialStart_startInd = trialStart_endInd
        
        
        
        
        
        
        
        # starting pupil size - start and end are reversed, because we come from the end
        trialEnd_endInd = EventTrialsInd['end'][ind]
        trialEnd_startInd = trialEnd_endInd - TrialComputationCheckInd
        trial_nanValues = NanValues[trialEnd_startInd:trialEnd_endInd]
        
        
        overlapping_time = TrialComputationTime/2 #s
        nIterations = int((TrialComputationCheckTime)/(TrialComputationTime-overlapping_time))-1
        
        done = 0
        
        
        
        while not done:
            
            iteration = nIterations-1
            while iteration > 0:
                NanValuesPresent = trial_nanValues[int(iteration*(TrialComputationTime-overlapping_time)*\
                        sampling_frequency) : int((iteration*(TrialComputationTime-overlapping_time) + \
                        TrialComputationTime)*sampling_frequency)]
                
                if sum(NanValuesPresent) < 1 and len(NanValuesPresent)>1:
                    
                    TrialEnd_noNanStartInd = trialEnd_startInd+int(iteration*(TrialComputationTime-overlapping_time)*\
                        sampling_frequency) 
                    TrialEnd_noNanEndInd = trialEnd_startInd+int((iteration*(TrialComputationTime-overlapping_time) + \
                        TrialComputationTime)*sampling_frequency)
                    done = 1
                    pupilSize_TrialEnd.append(np.mean(PupilMean_df['pupilMean_weighted'][TrialEnd_noNanStartInd:\
                                                                                             TrialEnd_noNanEndInd]))
                    break
            
                iteration = iteration - 1
            
            
            extraTrialEnd_startInd = trialEnd_startInd - TrialComputationCheckInd
            
            trial_nanValues = NanValues[extraTrialEnd_startInd:trialEnd_startInd]
            trialEnd_endInd = trialEnd_startInd
            trialEnd_startInd = extraTrialEnd_startInd
        
        
    
    return pupilSize_TrialStart, pupilSize_TrialEnd

In [29]:
class DataForEverySession:
    subjectID = ''
    blockNumber = ''
    sessionNumber = ''
    variable = ''
    dataForTrial = ''
    resultPathName = ''
   
    
    def printInfo(self):
        dataFrame = pd.DataFrame(list(zip([self.subjectID], [self.blockNumber], [self.sessionNumber], [self.dataForTrial])), columns=['subjectID', 'block', 'session', self.variable])
        
        return dataFrame
    
    def AddToFile(self):
        
        dataFrame = pd.DataFrame(list(zip([self.subjectID], [self.blockNumber], [self.sessionNumber], [self.dataForTrial])), columns=['subjectID', 'block', 'session', self.variable])
        
        book = load_workbook(self.resultPathName)
        writer = pd.ExcelWriter(self.resultPathName, engine='openpyxl')
        writer.book = book
        writer.sheets = dict((ws.title, ws) for ws in book.worksheets)
        startrow = writer.sheets['Sheet1'].max_row
        dataFrame.to_excel(writer, startrow = startrow, index = False, header = False)
        
        writer.save()
    

In [30]:
metricComputed_baseline = 'PupilSizeBaseline_300ms'
metricComputed_readingStart = 'PupilSize_readingStarting_300ms'
metricComputed_readingEnd = 'PupilSize_readingEnding_300ms'
metricComputed_writingStart = 'PupilSize_writingStarting_300ms'
metricComputed_writingEnd = 'PupilSize_writingEnding_300ms'
#metricComputed_correlation = 'PupilSizeCorrelation_300ms'


dataFolderName = r'E:\Data\Data' # accessing external hard disk with the data
a = re.compile('(?<=Data\\\\Data\\\\)(.*)(?=\\\\[1-9])')
subjectName_listElement = 3

#dataFolderName = r'C:\DTU\Data\201901_JanuaryExpt' # accessing data saved in the computer
#a = re.compile('(?<=Data\\\\201901_JanuaryExpt\\\\)(.*)(?=\\\\[1-9])')
#subjectName_listElement = 4

resultFileName_baseline = r'C:\DTU\Data\201901_JanuaryExpt\DataExtracted\IndividualTrials\Subject_Block_Session_Trial_' \
+ metricComputed_baseline +  '.xlsx'

resultFileName_readingStart = r'C:\DTU\Data\201901_JanuaryExpt\DataExtracted\IndividualTrials\Subject_Block_Session_Trial_' \
+ metricComputed_readingStart +  '.xlsx'
resultFileName_readingEnd = r'C:\DTU\Data\201901_JanuaryExpt\DataExtracted\IndividualTrials\Subject_Block_Session_Trial_' \
+ metricComputed_readingEnd +  '.xlsx'

resultFileName_writingStart = r'C:\DTU\Data\201901_JanuaryExpt\DataExtracted\IndividualTrials\Subject_Block_Session_Trial_' \
+ metricComputed_writingStart +  '.xlsx'
resultFileName_writingEnd = r'C:\DTU\Data\201901_JanuaryExpt\DataExtracted\IndividualTrials\Subject_Block_Session_Trial_' \
+ metricComputed_writingEnd +  '.xlsx'

#resultFileName_correlation = r'C:\DTU\Data\201901_JanuaryExpt\DataExtracted\IndividualSessions_TrialAvg\Subject_Block_Session_' \
#+ metricComputed_correlation +  '.xlsx'


for root, dirs, subfolder in os.walk(dataFolderName):
    
    technique = 'dwell_time'
    
    if not dirs:
        
        #if 'notCompleted' in root or 'notInclude' in root: # Some subjects do not have gaze log and have been marked as 
        
        if 'noData' in root or 'Trial' in root or 'trial' in root or 'Nothing' in root: # Some subjects do not have gaze 
            # log and have been marked as notInclude
            continue
        if 'Jonas' in root or 'Praktikant' in root or 'Villads' in root:
            continue
            
        #if 'ac\\3_MS\\' not in root:
        #    continue
        if 'Picture' in root:
            continue
        #if '_MS' not in root:
        #    continue
        
        #if 'bh2\\3\\2019-02-28-16-47-35_1' not in root:
        #    continue
        
        
        
        #if '2019-1-16-16-36-17_1stPart_2' not in root:
        #    continue
            
        keysSelected = None
        userKeys = None
        phraseLog = None
        
        for file in subfolder:
            
            
            if fnmatch.fnmatch(file, 'KeySelection*'):
                try:
                    
                    fKeysSelected = open(root + '\\' + file, encoding='utf-8', newline='')
                    readerKeysSelected = csv.reader(fKeysSelected)
                    keysSelected = list(readerKeysSelected)
                    
                    keysSelected.remove(keysSelected[0])
                except:
                    if fKeysSelected is not None:
                        fKeysSelected.close()
                    else:
                        print('error in opening the KeySelection log file')
                        
            
            if fnmatch.fnmatch(file, 'user*'):
                try:
                    fUserKey = open(root + '\\' + file, encoding='utf-8',  newline='')
                    readerUserKey = csv.reader(fUserKey, quotechar=None)
                    userKeys = list(readerUserKey)
                    userKeys.remove(userKeys[0])
                except:
                    if fUserKey is not None:
                        fUserKey.close()
                    else:
                        print('error in opening the user key log file')
                        
            if fnmatch.fnmatch(file, 'phrase*'):
                try:
                    fPhraseLog = open(root + '\\' + file, encoding='utf-8')
                    readerPhraseLog = csv.reader(fPhraseLog, quotechar=None)
                    phraseLog = list(readerPhraseLog)
                    
                except:
                    if fPhraseLog is not None:
                        fPhraseLog.close()
                    else:
                        print('error in opening the phrase log file')
                        
            if fnmatch.fnmatch(file, 'multiKey*'):
                technique = 'multiKey_selection'
            
            if fnmatch.fnmatch(file, 'tobiiGazeLog*'):
                try:
                    fGazeLog = open(root + '\\' + file, encoding='utf-8', newline='')
                    readerGazeLog = csv.reader(fGazeLog, quotechar=None)
                    gazeLog = list(readerGazeLog)
                    gazeLog.remove(gazeLog[0])
                    gazeLog.remove(gazeLog[-1])
                    
                except:
                    if fGazeLog is not None:
                        fGazeLog.close()
                    else:
                        print('error in opening the scratchpad log file')
            
                    
                     
        if keysSelected is None or userKeys is None or phraseLog is None or gazeLog is None:
            continue
        else:
                
            print('subject path', root)
            subjAndSessionName = '__'.join(root.split('\\')[subjectName_listElement:])
            subjName = subjAndSessionName.split('__')[0]
            print('subject and session name: ', subjAndSessionName)
            sessionFolderName = root.split('\\')[-1]
            
            
            # fix phraselog due to comma related file changes
            phraseLog_new = FixScratchPad(phraseLog)
            
            # fix userKeys due to comma related file changes
            userKeys_new = FixUserKeys(userKeys)
            
            
            # need to fix keys selected, where rows get combined because of an inverted comma
            keysSelected_new = FixKeysSelected(keysSelected)
            
            # find start time of typing
            timeTyping = OptiKeyTypingTime(userKeys_new)
            
            timeBaselineQuestion = getBaselineQuestion(timeTyping, keysSelected_new)
            
            # divide complete data into epochs of phrases
            timeStartEndDict_reading, timeStartEndDict_writing = FindTrialTimes(keysSelected_new, timeTyping, root)
            
            if sessionFolderName in dict_noGazeData:
                print('no gaze data present')
                pupilSize_baseline = [np.nan]*len(timeStartEndDict_reading['start'])
                
                pupilSize_starting_reading = [np.nan]*len(timeStartEndDict_writing['start'])
                pupilSize_ending_reading = [np.nan]*len(timeStartEndDict_writing['start'])
                pupilSize_starting_writing = [np.nan]*len(timeStartEndDict_writing['start'])
                pupilSize_ending_writing = [np.nan]*len(timeStartEndDict_writing['start'])
                
            else:
            
               # filter the data
                pupilMean_df, nanValues, correlationRL = FilterPupilSize_wWeightedMean(gazeLog, timeTyping, subjAndSessionName)
            
                # find a vector indicating True for the start and end of trials, False otherwise
                eventTrials_reading, eventTrials_readingIndex = DivideTimeIntoTrials(pupilMean_df, \
                                                                                     timeStartEndDict_reading)
                eventTrials_writing, eventTrials_writingIndex = DivideTimeIntoTrials(pupilMean_df, \
                                                                                     timeStartEndDict_writing)
                
                # divide trials into reading and writing
                eventReading, eventReading_index = FindReadingPartsOfTrial(eventTrials_readingIndex, keysSelected_new, pupilMean_df, root)
                eventWriting, eventWriting_index = FindWritingPartsOfTrial(eventTrials_writingIndex, keysSelected_new, pupilMean_df, eventReading_index)
                
                
                # baseline time extraction
                #pupilSize_baseLine = GetBaseline(eventTrials_reading, pupilData_filtered)
                timeBaseline = 0.3 # seconds
                pupilSize_baseline = GetBaselineForEveryTrial(eventReading_index, pupilMean_df, nanValues, timeBaseline, \
                                  timeBaselineQuestion, subjAndSessionName)
                
                #print(pupilSize_baseline)
                # compute the pupil size for the trialComputationTime for reading and writing, at the beginning and end
                # of the parts
                trialComputationTime = 0.3                
                
                pupilSize_starting_reading, pupilSize_ending_reading = GetPupilSize_startingAndEnding(\
                                eventReading_index, pupilMean_df, trialComputationTime, nanValues)
                
                pupilSize_starting_writing, pupilSize_ending_writing = GetPupilSize_startingAndEnding(\
                                eventWriting_index, pupilMean_df, trialComputationTime, nanValues)
                
            
                
            if '1stPart' in root:
                print('1stPart')
                pupilSize_baseline1 = pupilSize_baseline
                pupilSize_starting_reading1 = pupilSize_starting_reading
                pupilSize_ending_reading1 = pupilSize_ending_reading
                pupilSize_starting_writing1 = pupilSize_starting_writing
                pupilSize_ending_writing1 = pupilSize_ending_writing
                
                """
                dataToSave_correlationRL = DataForEverySession()
                dataToSave_correlationRL.subjectID = subjAndSessionName.split('__')[0]
                dataToSave_correlationRL.blockNumber = subjAndSessionName.split('__')[1]
                dataToSave_correlationRL.sessionNumber = subjAndSessionName[-1]
                dataToSave_correlationRL.variable = metricComputed_correlation
                dataToSave_correlationRL.dataForTrial = correlationRL
                dataToSave_correlationRL.resultPathName = resultFileName_correlation
            
                print(dataToSave_correlationRL.printInfo())
                dataToSave_correlationRL.AddToFile()
                """
                continue
            
            
            if '2ndPart' in root:
                print('2ndPart')
                pupilSize_baseline2 = pupilSize_baseline
                
                pupilSize_starting_reading2 = pupilSize_starting_reading
                pupilSize_ending_reading2 = pupilSize_ending_reading
                pupilSize_starting_writing2 = pupilSize_starting_writing
                pupilSize_ending_writing2 = pupilSize_ending_writing
                
                pupilSize_baseline = pupilSize_baseline1 + pupilSize_baseline2
                
                pupilSize_starting_reading = pupilSize_starting_reading1 + pupilSize_starting_reading2
                pupilSize_ending_reading = pupilSize_ending_reading1 + pupilSize_ending_reading2
                pupilSize_starting_writing = pupilSize_starting_writing1 + pupilSize_starting_writing2
                pupilSize_ending_writing = pupilSize_ending_writing1 + pupilSize_ending_writing2
                
                pupilSize_baseline1 = list()
                pupilSize_starting_reading1 = list()
                pupilSize_ending_reading1 = list()
                pupilSize_starting_writing1 = list()
                pupilSize_ending_writing1 = list()
                
                
            print('')
            # save the baseline pupil size for every 
            dataToSave_baseline = DataForEveryTrial()
            dataToSave_baseline.subjectID = subjAndSessionName.split('__')[0]
            dataToSave_baseline.blockNumber = subjAndSessionName.split('__')[1]
            dataToSave_baseline.sessionNumber = subjAndSessionName[-1]
            dataToSave_baseline.variable = metricComputed_baseline
            dataToSave_baseline.dataForTrial = pupilSize_baseline
            dataToSave_baseline.resultPathName = resultFileName_baseline
            
            print(dataToSave_baseline.printInfo())
            
            
            # save the reading start
            dataToSave_readingStart = DataForEveryTrial()
            dataToSave_readingStart.subjectID = subjAndSessionName.split('__')[0]
            dataToSave_readingStart.blockNumber = subjAndSessionName.split('__')[1]
            dataToSave_readingStart.sessionNumber = subjAndSessionName[-1]
            dataToSave_readingStart.variable = metricComputed_readingStart
            dataToSave_readingStart.dataForTrial = pupilSize_starting_reading
            dataToSave_readingStart.resultPathName = resultFileName_readingStart
            
            print(dataToSave_readingStart.printInfo())
            
            
            # save the reading end
            dataToSave_readingEnd = DataForEveryTrial()
            dataToSave_readingEnd.subjectID = subjAndSessionName.split('__')[0]
            dataToSave_readingEnd.blockNumber = subjAndSessionName.split('__')[1]
            dataToSave_readingEnd.sessionNumber = subjAndSessionName[-1]
            dataToSave_readingEnd.variable = metricComputed_readingEnd
            dataToSave_readingEnd.dataForTrial = pupilSize_ending_reading
            dataToSave_readingEnd.resultPathName = resultFileName_readingEnd
            
            print(dataToSave_readingEnd.printInfo())
            
            
            # save the writing start
            dataToSave_writingStart = DataForEveryTrial()
            dataToSave_writingStart.subjectID = subjAndSessionName.split('__')[0]
            dataToSave_writingStart.blockNumber = subjAndSessionName.split('__')[1]
            dataToSave_writingStart.sessionNumber = subjAndSessionName[-1]
            dataToSave_writingStart.variable = metricComputed_writingStart
            dataToSave_writingStart.dataForTrial = pupilSize_starting_writing
            dataToSave_writingStart.resultPathName = resultFileName_writingStart
            
            print(dataToSave_writingStart.printInfo())
            
            
            # save the writing start
            dataToSave_writingEnd = DataForEveryTrial()
            dataToSave_writingEnd.subjectID = subjAndSessionName.split('__')[0]
            dataToSave_writingEnd.blockNumber = subjAndSessionName.split('__')[1]
            dataToSave_writingEnd.sessionNumber = subjAndSessionName[-1]
            dataToSave_writingEnd.variable = metricComputed_writingEnd
            dataToSave_writingEnd.dataForTrial = pupilSize_ending_writing
            dataToSave_writingEnd.resultPathName = resultFileName_writingEnd
            
            print(dataToSave_writingEnd.printInfo())
            
            
            
            dataToSave_baseline.AddToFile()
            dataToSave_readingStart.AddToFile()
            dataToSave_readingEnd.AddToFile()
            dataToSave_writingStart.AddToFile()
            dataToSave_writingEnd.AddToFile()
            
            
            """
            
            dataToSave_correlationRL = DataForEverySession()
            dataToSave_correlationRL.subjectID = subjAndSessionName.split('__')[0]
            dataToSave_correlationRL.blockNumber = subjAndSessionName.split('__')[1]
            dataToSave_correlationRL.sessionNumber = subjAndSessionName[-1]
            dataToSave_correlationRL.variable = metricComputed_correlation
            dataToSave_correlationRL.dataForTrial = correlationRL
            dataToSave_correlationRL.resultPathName = resultFileName_correlation
            
            print(dataToSave_correlationRL.printInfo())
            dataToSave_correlationRL.AddToFile()
            """
            
            
            print('')
            
            

subject path E:\Data\Data\ac\1\2019-02-11-11-18-30_1
subject and session name:  ac__1__2019-02-11-11-18-30_1
[[1.         0.88463318]
 [0.88463318 1.        ]]

  subjectID block session  trial  PupilSizeBaseline_300ms
0        ac     1       1      0                 4.205012
1        ac     1       1      1                 4.251083
2        ac     1       1      2                 4.398345
3        ac     1       1      3                 4.215148
4        ac     1       1      4                 4.239304
  subjectID block session  trial  PupilSize_readingStarting_300ms
0        ac     1       1      0                         4.205492
1        ac     1       1      1                         4.251083
2        ac     1       1      2                         4.399323
3        ac     1       1      3                         4.190758
4        ac     1       1      4                         4.192430
  subjectID block session  trial  PupilSize_readingEnding_300ms
0        ac     1       1      

subject path E:\Data\Data\ac\3_MS\2019-02-14-14-28-49_1
subject and session name:  ac__3_MS__2019-02-14-14-28-49_1
[[1.         0.88600074]
 [0.88600074 1.        ]]

  subjectID block session  trial  PupilSizeBaseline_300ms
0        ac  3_MS       1      0                 4.164857
1        ac  3_MS       1      1                 4.445799
2        ac  3_MS       1      2                 4.182652
3        ac  3_MS       1      3                 4.307734
4        ac  3_MS       1      4                 4.239622
  subjectID block session  trial  PupilSize_readingStarting_300ms
0        ac  3_MS       1      0                         4.597075
1        ac  3_MS       1      1                         3.972125
2        ac  3_MS       1      2                         4.184029
3        ac  3_MS       1      3                         4.310649
4        ac  3_MS       1      4                         4.242049
  subjectID block session  trial  PupilSize_readingEnding_300ms
0        ac  3_MS       1

subject path E:\Data\Data\ac\5\2019-02-19-15-07-5_1
subject and session name:  ac__5__2019-02-19-15-07-5_1
[[1.         0.80941898]
 [0.80941898 1.        ]]

  subjectID block session  trial  PupilSizeBaseline_300ms
0        ac     5       1      0                 4.386149
1        ac     5       1      1                 4.580319
2        ac     5       1      2                 4.278393
3        ac     5       1      3                 4.412755
4        ac     5       1      4                 4.419640
  subjectID block session  trial  PupilSize_readingStarting_300ms
0        ac     5       1      0                         4.445468
1        ac     5       1      1                         4.505337
2        ac     5       1      2                         4.278393
3        ac     5       1      3                         4.493995
4        ac     5       1      4                         4.419640
  subjectID block session  trial  PupilSize_readingEnding_300ms
0        ac     5       1      0 

subject path E:\Data\Data\af\2\2019-01-17-15-03-40_1
subject and session name:  af__2__2019-01-17-15-03-40_1
[[1.         0.92118671]
 [0.92118671 1.        ]]

  subjectID block session  trial  PupilSizeBaseline_300ms
0        af     2       1      0                 4.229691
1        af     2       1      1                 4.115266
2        af     2       1      2                 4.510297
3        af     2       1      3                 3.881711
4        af     2       1      4                 4.337184
  subjectID block session  trial  PupilSize_readingStarting_300ms
0        af     2       1      0                         4.196646
1        af     2       1      1                         4.115266
2        af     2       1      2                         4.505426
3        af     2       1      3                         3.854343
4        af     2       1      4                         4.341710
  subjectID block session  trial  PupilSize_readingEnding_300ms
0        af     2       1      

subject path E:\Data\Data\af\4\2019-02-12-11-07-43_1
subject and session name:  af__4__2019-02-12-11-07-43_1
[[1.         0.90030391]
 [0.90030391 1.        ]]

  subjectID block session  trial  PupilSizeBaseline_300ms
0        af     4       1      0                 4.791109
1        af     4       1      1                 4.981246
2        af     4       1      2                 5.150588
3        af     4       1      3                 5.286998
4        af     4       1      4                 4.546694
  subjectID block session  trial  PupilSize_readingStarting_300ms
0        af     4       1      0                         4.756679
1        af     4       1      1                         4.985127
2        af     4       1      2                         5.106756
3        af     4       1      3                         5.303735
4        af     4       1      4                         4.510575
  subjectID block session  trial  PupilSize_readingEnding_300ms
0        af     4       1      

...


subject path E:\Data\Data\ph\4\2019-02-18-16-19-28_2
subject and session name:  ph__4__2019-02-18-16-19-28_2
[[1.         0.95561806]
 [0.95561806 1.        ]]

  subjectID block session  trial  PupilSizeBaseline_300ms
0        ph     4       2      0                 5.727248
1        ph     4       2      1                 5.563206
2        ph     4       2      2                 5.575732
3        ph     4       2      3                 5.362808
4        ph     4       2      4                 4.969092
  subjectID block session  trial  PupilSize_readingStarting_300ms
0        ph     4       2      0                         5.727248
1        ph     4       2      1                         5.491541
2        ph     4       2      2                         5.612094
3        ph     4       2      3                         5.371173
4        ph     4       2      4                         4.987559
  subjectID block session  trial  PupilSize_readingEnding_300ms
0        ph     4       2     


subject path E:\Data\Data\rh\2\2019-03-13-13-03-17_1
subject and session name:  rh__2__2019-03-13-13-03-17_1
[[1.         0.93735964]
 [0.93735964 1.        ]]

  subjectID block session  trial  PupilSizeBaseline_300ms
0        rh     2       1      0                 4.351467
1        rh     2       1      1                 5.202222
2        rh     2       1      2                 5.074492
3        rh     2       1      3                 4.993048
4        rh     2       1      4                 5.095618
  subjectID block session  trial  PupilSize_readingStarting_300ms
0        rh     2       1      0                         4.351467
1        rh     2       1      1                         5.146440
2        rh     2       1      2                         5.071797
3        rh     2       1      3                         4.983551
4        rh     2       1      4                         5.096253
  subjectID block session  trial  PupilSize_readingEnding_300ms
0        rh     2       1     


subject path E:\Data\Data\rh\4\2019-04-11-13-40-31_2
subject and session name:  rh__4__2019-04-11-13-40-31_2
[[1.         0.93850755]
 [0.93850755 1.        ]]

  subjectID block session  trial  PupilSizeBaseline_300ms
0        rh     4       2      0                 4.997802
1        rh     4       2      1                 4.929872
2        rh     4       2      2                 5.232627
3        rh     4       2      3                 5.203611
4        rh     4       2      4                 5.178242
  subjectID block session  trial  PupilSize_readingStarting_300ms
0        rh     4       2      0                         5.000842
1        rh     4       2      1                         5.032091
2        rh     4       2      2                         5.233253
3        rh     4       2      3                         5.202494
4        rh     4       2      4                         5.177380
  subjectID block session  trial  PupilSize_readingEnding_300ms
0        rh     4       2     


subject path E:\Data\Data\ys\2\2019-01-16-15-18-50_1stPart_1
subject and session name:  ys__2__2019-01-16-15-18-50_1stPart_1
[[1.         0.93284464]
 [0.93284464 1.        ]]
1stPart
subject path E:\Data\Data\ys\2\2019-01-16-15-42-51_2ndPart_1
subject and session name:  ys__2__2019-01-16-15-42-51_2ndPart_1
[[1.         0.95630344]
 [0.95630344 1.        ]]
2ndPart

  subjectID block session  trial  PupilSizeBaseline_300ms
0        ys     2       1      0                 5.551479
1        ys     2       1      1                 5.766089
2        ys     2       1      2                 5.312633
3        ys     2       1      3                 5.328384
4        ys     2       1      4                 4.112588
  subjectID block session  trial  PupilSize_readingStarting_300ms
0        ys     2       1      0                         5.602731
1        ys     2       1      1                         5.765112
2        ys     2       1      2                         5.312633
3        ys     2 


subject path E:\Data\Data\ys\4_MS\2019-01-30-11-22-25_1
subject and session name:  ys__4_MS__2019-01-30-11-22-25_1
[[1.         0.93702794]
 [0.93702794 1.        ]]

  subjectID block session  trial  PupilSizeBaseline_300ms
0        ys  4_MS       1      0                 5.621048
1        ys  4_MS       1      1                 5.199656
2        ys  4_MS       1      2                 5.150062
3        ys  4_MS       1      3                 5.455159
4        ys  4_MS       1      4                 5.066239
  subjectID block session  trial  PupilSize_readingStarting_300ms
0        ys  4_MS       1      0                         5.620265
1        ys  4_MS       1      1                         5.211290
2        ys  4_MS       1      2                         5.150472
3        ys  4_MS       1      3                         5.382867
4        ys  4_MS       1      4                         4.944702
  subjectID block session  trial  PupilSize_readingEnding_300ms
0        ys  4_MS       